# Fast L-BFGS and VL-BFGS on GPU

## 0. MODULE

In [ ]:
# Computation
import numpy as np
import torch

from scipy.optimize import fmin_l_bfgs_b

# Random
from torch.distributions.normal import Normal

# Perso
from common import *
from VL_BFGS import *
from feature_engineering import *

# Plot
import matplotlib.pyplot as plt

## 1. DONNÉES SIMULÉES

### 1.1 Vérification de la convergence

Nous allons commencer par vérifier la convergence de nos deux algorithmes en les comparant à l'implémentation Scipy. 

Pour cela nous simulons un petit jeu de données pour une régression linéaire.

In [ ]:
n_features = 1000
n = Normal(0, 1)
coefs = n.sample(torch.Size([n_features]))

lbda = 0.1
w0 = torch.zeros(n_features)

f = linear_loss
f_grad = linear_grad

X, y = simulate_data(coefs, n_samples=50000, for_logreg=False)

Scipy n'accepte que des array numpy en input, pour cela nous transformons nos torch tensor. Nous utilisons aussi des fonctions codées en numpy.

In [ ]:
w0_numpy = w0.numpy()

f_numpy = linear_loss_numpy
f_grad_numpy = linear_grad_numpy

X_numpy, y_numpy = X.numpy(), y.numpy()

In [ ]:
# Scipy
_, f_min_scipy, _ = fmin_l_bfgs_b(f_numpy, w0_numpy, f_grad_numpy, args=(X_numpy, y_numpy, lbda))

# Notre implémentation de L-BFGS
optimizer = lbfgs(f, f_grad, m=10, vector_free=False, device='cpu')
_, f_min, _, _ = optimizer.fit(X, y, w0, lbda)

# Notre implémentation de VL-BFGS
optimizer_VL = lbfgs(f, f_grad, m=10, vector_free=True, device='cpu')
_, f_min_VL, _, _ = optimizer.fit(X, y, w0, lbda)

Nous comparons enfin les loss obtenues par les 3 algorithmes.

In [ ]:
print('Scipy: %.4f' % (f_min_scipy))
print('L-BFGS: %.4f' % (f_min[-1]))
print('VL-BFGS: %.4f' % (f_min_VL[-1]))

Nos deux implémentations convergent donc bien, nous allons donc maintenant pouvoir nous intéresser à leur vitesse de convergence.

In [ ]:
del X_numpy, y_numpy

### 1.2 Convergence et temps de calcul

In [ ]:
n_samples = 200000
n_features = 3000

n = Normal(0, 1)
coefs = n.sample(torch.Size([n_features]))

lbda = 0.1

#### 1.2.1 Regression linéaire

In [ ]:
X, y = simulate_data(coefs, n_samples, for_logreg=False)

In [ ]:
w0 = torch.zeros(n_features)
f = linear_loss
f_grad = linear_grad

In [ ]:
# CPU
optimizer = lbfgs(f, f_grad, m=10, vector_free=False, device='cpu')
_, objectives_cpu, computing_time_cpu, _ = optimizer.fit(X, y, w0, lbda)

# GPU
optimizer = lbfgs(f, f_grad, m=10, vector_free=False, device='cuda:0')
_, objectives_gpu, computing_time_gpu, _ = optimizer.fit(X, y, w0, lbda)

# GPU VL
optimizer = lbfgs(f, f_grad, m=10, vector_free=True, device='cuda:0')
_, objectives_gpu_vl, computing_time_gpu_vl, _ = optimizer.fit(X, y, w0, lbda)

In [ ]:
f_min = min(objectives_gpu[-1], objectives_cpu[-1], objectives_gpu_vl[-1])

In [ ]:
plt.figure(figsize=(15, 5))
plt.semilogy(np.linspace(0, computing_time_cpu, len(objectives_cpu)), objectives_cpu - f_min, lw=2, label='CPU')
plt.semilogy(np.linspace(0, computing_time_gpu, len(objectives_gpu)), objectives_gpu - f_min, lw=2, label='GPU')
plt.semilogy(np.linspace(0, computing_time_gpu_vl, len(objectives_gpu_vl)), 
             objectives_gpu_vl - f_min, lw=2, label='GPU (vector-free)')
plt.yscale('log')
plt.title("Linear Regression", fontsize=16)
plt.legend(title='Device', loc='best')
plt.xlabel('Time (seconds)')
plt.ylabel('Distance to minimum')
plt.show()

#### 1.2.2 Régression logistique

In [ ]:
y = torch.sign(y)

In [ ]:
w0 = torch.zeros(n_features)
f = logistic_loss
f_grad = logistic_grad

In [ ]:
# CPU
optimizer = lbfgs(f, f_grad, m=10, vector_free=False, device='cpu')
_, objectives_cpu, computing_time_cpu, _ = optimizer.fit(X, y, w0, lbda)

# GPU
optimizer = lbfgs(f, f_grad, m=10, vector_free=False, device='cuda:0')
_, objectives_gpu, computing_time_gpu, _ = optimizer.fit(X, y, w0, lbda)

# GPU VL
optimizer = lbfgs(f, f_grad, m=10, vector_free=True, device='cuda:0')
_, objectives_gpu_vl, computing_time_gpu_vl, _ = optimizer.fit(X, y, w0, lbda)

In [ ]:
f_min = min(objectives_gpu[-1], objectives_cpu[-1], objectives_gpu_vl[-1])

In [ ]:
plt.figure(figsize=(15, 5))
plt.semilogy(np.linspace(0, computing_time_cpu, len(objectives_cpu)), objectives_cpu - f_min, lw=2, label='CPU')
plt.semilogy(np.linspace(0, computing_time_gpu, len(objectives_gpu)), objectives_gpu - f_min, lw=2, label='GPU')
plt.semilogy(np.linspace(0, computing_time_gpu_vl, len(objectives_gpu_vl)), 
             objectives_gpu_vl - f_min, lw=2, label='GPU (vector-free)')
plt.yscale('log')
plt.title("Logistic Regression", fontsize=16)
plt.legend(title='Device', loc='best')
plt.xlabel('Time (seconds)')
plt.ylabel('Distance to minimum')
plt.show()

Le temps de calcul est beaucoup moins important sur GPU que sur CPU dans le cas des deux algorithmes considérés.

Néanmoins VL-BFGS semble légérement moins rapide que L-BFGS. Cela est normal puisque le premier a d'abord été pensé pour être distribué et scalable du point de vue des features. Il serait donc plus rapide que le second sur un plus grand nombre de GPUs et pour un plus grand nombre de variables (que nous n'avons pu atteindre dû à une ram insuffisante).

Dans cette première comparaison nous n'avons pas pris en compte le temps de communication entre le CPU et le GPU, ce qui nuit à la performance globale des algorithmes.

### 1.3 Temps d'exécution en fonction de la dimension

In [ ]:
f = logistic_loss
f_grad = logistic_grad

n_features = np.arange(200, 10000, 200)
n_samples = 50000

cpu_time, gpu_time, gpu_vl_time = [], [], []

for n_feature in n_features:
    
    n = Normal(0, 1)
    coefs = n.sample(torch.Size([n_feature]))
    w0 = torch.zeros(n_feature)
    X, y = simulate_data(coefs, n_samples=50000, for_logreg=True)
    
    # CPU
    optimizer = lbfgs(f, f_grad, m=10, vector_free=False, device='cpu')
    _, _, computing_time_cpu, com_time_cpu = optimizer.fit(X, y, w0, lbda)
    
    # GPU
    optimizer = lbfgs(f, f_grad, m=10, vector_free=False, device='cuda:0')
    _, _, computing_time_gpu, com_time_gpu = optimizer.fit(X, y, w0, lbda)
    
    # GPU VL
    optimizer = lbfgs(f, f_grad, m=10, vector_free=True, device='cuda:0')
    _, _, computing_time_gpu_vl, com_time_gpu_vl = optimizer.fit(X, y, w0, lbda)
    
    cpu_time.append(computing_time_cpu + com_time_cpu)
    gpu_time.append(computing_time_gpu + com_time_gpu)
    gpu_vl_time.append(computing_time_gpu_vl + com_time_gpu_vl)

In [ ]:
plt.figure(figsize=(15, 5))
plt.title('Runtime comparison', fontsize=16)
plt.plot(n_features, cpu_time, label='CPU')
plt.plot(n_features, gpu_time, label='GPU')
plt.plot(n_features, gpu_vl_time, label='GPU (vector-free)')
plt.legend(title='Device', loc='best')
plt.ylabel('Runtime (s)')
plt.xlabel('Dimension')
plt.show()

### 1.4 Comparaison détaillée

Dans cette partie nous allons regarder précisément le temps de communication entre le CPU et le GPU (le temps nécessaire pour que les données soient envoyées au GPU) et le temps de calcul. Nous allons exécuter chaque algorithme 10x, et ce pour plusieurs datasets (toujours simulés) afin d'obtenir des statistiques plus représentatives.

In [ ]:
!python simulation.py

## 2. DONNÉES RÉELLES 

Dans cette partie nous allons appliquer nos deux algorithmes à un jeu de données réelles.

### 2.1 Données

Le dataset comporte 33 variables, lesquelles sont différentes mesures physiques concernant l'activité solaire. Il y a environ 500000 observations, chacune représentant un instant t. Il s'agit de prédire, pour chaque instant t, la présence d'une tempête solaire.

Il s'agit d'une tache de classification pour laquelle nous pouvons utiliser une regression logistique. Cependant ce modèle ne permet pas naturellement d'assimiler la relation temporelle entre les différentes observations. Afin de palier ce défaut nous allons faire un important feature engineering afin de créer de nombreuses variables "temporelles". Nous allons par exemple créer une variable valant, pour l'observation t et le variable v, la moyenne de la variable v sur l'intervalle [t-1h, t]. 

En faisant ça pour toutes les variables, en utilisant différentes mesures (moyenne, médiane, variance, minimum, maximum) et sur plusieurs intervalles temporels (1h, 3h, 5h, 10h, 20h) nous obtenons un total de 1749.

In [ ]:
print('>> Loading data')
X = np.load('data/data_train.npy')
y = pd.read_csv('data/labels_train.csv')

print('>> To tensor')
X = torch.tensor(X, dtype=torch.float)
y = torch.tensor(y.values, dtype=torch.float).squeeze()

print('>> Training on %2d samples and %2d features' % X.size())

### 2.2 Comparaison

In [ ]:
# Parameters
lbda = 0.1

# Initialization
w0 = torch.zeros(X.size(1))
f = logistic_loss
f_grad = logistic_grad

In [ ]:
# CPU
optimizer = lbfgs(f, f_grad, m=10, vector_free=False, device='cpu')
_, objectives_cpu, computing_time_cpu, _ = optimizer.fit(X, y, w0, lbda)

# GPU
optimizer = lbfgs(f, f_grad, m=10, vector_free=False, device='cuda:0')
_, objectives_gpu, computing_time_gpu, _ = optimizer.fit(X, y, w0, lbda)

# GPU VL
optimizer = lbfgs(f, f_grad, m=10, vector_free=True, device='cuda:0')
_, objectives_gpu_vl, computing_time_gpu_vl, _ = optimizer.fit(X, y, w0, lbda)

In [ ]:
f_min = min(objectives_gpu[-1], objectives_cpu[-1], objectives_gpu_vl[-1])

In [ ]:
plt.figure(figsize=(15, 5))
plt.semilogy(np.linspace(0, computing_time_cpu, len(objectives_cpu)), objectives_cpu - f_min, lw=2, label='CPU')
plt.semilogy(np.linspace(0, computing_time_gpu, len(objectives_gpu)), objectives_gpu - f_min, lw=2, label='GPU')
plt.semilogy(np.linspace(0, computing_time_gpu_vl, len(objectives_gpu_vl)), 
             objectives_gpu_vl - f_min, lw=2, label='GPU (vector-free)')
plt.yscale('log')
plt.title("Real data", fontsize=16)
plt.legend(title='Device', loc='best')
plt.xlabel('Time (seconds)')
plt.ylabel('Distance to minimum')
plt.show()

## 3. CONCLUSION 